In [1]:
import os
import json
import requests
import glob

import pandas as pd

from dotenv import load_dotenv
from tqdm.notebook import tqdm
tqdm.pandas()

from IPython.display import Image

In [2]:
df = pd.read_csv('../data_amelia/processed/2014_protest_data.csv')
df.head()

,GLOBALEVENTID,SQLDATE,MonthYear,Year,FractionDate,Actor1Code,Actor1Name,Actor1CountryCode,Actor1KnownGroupCode,Actor1EthnicCode,...,Actor2Geo_FeatureID,ActionGeo_Type,ActionGeo_FullName,ActionGeo_CountryCode,ActionGeo_ADM1Code,ActionGeo_Lat,ActionGeo_Long,ActionGeo_FeatureID,DATEADDED,SOURCEURL
0,281510593,20140101,201401,2014,2014.0027,TUR,TURKEY,TUR,NaN,NaN,...,NaN,2,"Alaska, United States",US,USAK,61.3850,-152.2680,AK,20140101,"Zaman website, Istanbul/BBC Monitoring/(c) BBC"
1,281515425,20140101,201401,2014,2014.0027,USA,UNITED STATES,USA,NaN,NaN,...,NaN,3,"Upper Providence, Pennsylvania, United States",US,USPA,39.9551,-75.4230,1197544,20140101,http://www.montgomerynews.com/articles/2014/01...
2,281515426,20140101,201401,2014,2014.0027,USA,UNITED STATES,USA,NaN,NaN,...,NaN,3,"Valley Park, Pennsylvania, United States",US,USPA,40.1093,-75.1052,1207721,20140101,http://www.montgomerynews.com/articles/2014/01...
3,281515427,20140101,201401,2014,2014.0027,USA,ALASKA,USA,NaN,NaN,...,NaN,2,"Alaska, United States",US,USAK,61.3850,-152.2680,AK,20140101,"Zaman website, Istanbul/BBC Monitoring/(c) BBC"
4,281518684,20140101,201401,2014,2014.0027,USA,ALASKA,USA,NaN,NaN,...,-740445,2,"Alaska, United States",US,USAK,61.3850,-152.2680,AK,20140101,"Zaman website, Istanbul/BBC Monitoring/(c) BBC"


In [5]:
data_filepath = '../data_amelia/processed/'
file_list = glob.glob(f"{data_filepath}/*_protest_data.csv")

# Concatenate all DataFrames
combined_df = pd.concat([pd.read_csv(file, dtype={'SQLDATE': str}) for file in file_list], ignore_index=True)

# Getting dates into the correct form for consistency

combined_df['SQLDATE'] = pd.to_datetime(combined_df['SQLDATE'].astype(str), format='%Y%m%d').dt.strftime('%Y-%m-%d')

# Filtering then renaming to get relevant data and improve clarity
relevant_columns = ['GLOBALEVENTID', 'SQLDATE', 'Actor1CountryCode', 'SOURCEURL', 'GoldsteinScale', 'NumMentions', 'AvgTone']
protest_data = combined_df[relevant_columns].copy()
protest_data.rename(columns={'GLOBALEVENTID': 'EventID', 'SQLDATE': 'Date', 'Actor1CountryCode': 'RelevantCountry'}, inplace= True)

protest_data

,EventID,Date,RelevantCountry,SOURCEURL,GoldsteinScale,NumMentions,AvgTone
0,6.132572e+08,2017-01-01,NaN,http://wypr.org/?page=1901,-6.5,16,-2.656748
1,6.133204e+08,2017-01-01,USA,http://www.dailyitem.com/news/local_news/pensi...,-6.5,2,-3.514377
2,6.133279e+08,2017-01-01,USA,http://www.foxnews.com/politics/2015/05/08/uc-...,-6.5,9,-6.250000
3,6.133279e+08,2017-01-01,USA,http://www.foxnews.com/politics/2015/05/08/uc-...,-6.5,1,-6.250000
4,6.133821e+08,2017-01-01,NaN,http://www.azcentral.com/picture-gallery/news/...,-6.5,2,-1.166781
...,...,...,...,...,...,...,...
299775,1.021377e+09,2021-12-31,NaN,https://qctimes.com/news/state-and-regional/go...,-6.5,6,-3.088235
299776,1.021377e+09,2021-12-31,NaN,https://qctimes.com/news/state-and-regional/go...,-6.5,2,-3.088235
299777,1.021378e+09,2021-12-31,USA,https://qctimes.com/news/state-and-regional/go...,-6.5,2,-3.088235
299778,1.021387e+09,2021-12-31,USA,https://www.cbs19.tv/article/news/local/new-yo...,-6.5,6,4.074703


In [4]:
protest_data.to_csv('../data_amelia/processed/all_protest_data.csv', index=False)